In [36]:
#Config
import torch
#import torch.nn as nn
#import torch.optim as optim
#import torch.nn.functional as F
import numpy as np
#import torchrl
import random
from collections import Counter

import csv
from itertools import product
import copy

def load_hand_rankings(csv_path):
    """
    Loads the hand ranking CSV into a dictionary:
      dict[ (card1, card2, card3, card4, card5, flush_flag) ] = hand_rank
    Assumes each row has 7 columns:
      rank, c1, c2, c3, c4, c5, flush_flag
    """
    lookup = {}
    with open(csv_path, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        for row in reader:
            # Convert everything to integers
            row_ints = list(map(int, row))
            
            hand_rank = row_ints[0]
            cards = tuple(row_ints[1:6]) 
            flush_flag = row_ints[6]
            
            # Store in dictionary:
            # Key is (cards, flush_flag), Value is hand_rank
            lookup[(cards, flush_flag)] = hand_rank
    
    return lookup

# Load the poker hand table
hand_lookup = load_hand_rankings('C:\\Users\\natha\\OneDrive\\Documents\\GitHub\\VS-Code-Workspace\\SmallProjects\\Pytorch\\PokerHandTable.csv')


In [17]:
#Create a card class: 
class Card():
    suits = [1, 2, 3, 4]
    values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

    def __init__(self, suit, value):
        if suit not in Card.suits:
            raise ValueError(f"Invalid suit: {suit}")
        if value not in Card.values:
            raise ValueError(f"Invalid value: {value}")
        self.suit = suit
        self.value = value

    def __repr__(self):
        return f"{self.value} of {self.suit}"
    def __eq__(self, other):
        if isinstance(other, Card):
            return self.suit == other.suit and self.value == other.value
        return False

In [ ]:


'''
We want to simulate a card game environment
The rules of the game are as follows: 
1. The game is played with a standard 52-card deck.
2. There are 5 rounds, one player, and a dealer.
3. At the start of each round, the player can choose any subset of cards from the deck that they want to add to their hand if it is flipped over.
4. After this, the dealer will flip cards over one by one, until they reach a card that is in the subset chosen by the player.
5. That card is then added to the player's hand, while all the other cards flipped over by the dealer during that round are added to the dealer's hand.
6. This is repeated for 5 rounds, and the player can choose a new subset every round, after these rounds, the player should then have a 5 card poker hand. 
7. The dealer can have any number of cards in their hand, but the player must have exactly 5 cards.
8. However, if the dealer ends the game with less than 8 cards, they draw from the deck until they have 8 cards.
9. The dealer also wins ties.
10. The player wins if they have a better 5 card poker hand than the dealer, otherwise the dealer wins.
'''
class CardGameEnv():
    def __init__(self):
        self.deck = self.create_deck()
        self.round = 1
        self.player_hand = []
        self.opponent_hand = []

    def create_deck(self):
        return [Card(suit, value) for suit in Card.suits for value in Card.values]

    def shuffle_deck(self):
        random.shuffle(self.deck)
        
    def get_state(self):
        return {
            'round': self.round,
            'deck': self.deck,
            'player_hand': self.player_hand,
            'opponent_hand': self.opponent_hand
        }
        
    def reset(self):
        self.deck = self.create_deck()
        self.shuffle_deck()
        self.player_hand = []
        self.opponent_hand = []
        self.round = 1
        return self.get_state()
    
    def determine_choice(self):
        #For our strat, we want to chose all cards 9 or up for the first 2 rounds
        #After this, we will choose the ranks of any cards in the player's hand
        
        if self.round <= 2:
            player_choice = [Card(suit, value) for suit in Card.suits for value in range(9, 15)]
            
        else:
            # after round 2, look at how many distinct ranks you hold
            hand_ranks = {c.value for c in self.player_hand}
            
            if len(hand_ranks) == 1:
                # only one rank in hand → fall back to any 9+
                player_choice = [card for card in self.deck if card.value >= 9]
            else:
                # more than one rank → take any deck card matching one of your ranks
                player_choice = [card for card in self.deck if card.value in hand_ranks]
        
        return player_choice
    
    #Now we will write the simulator to run the game 
    def simulate_game(self):
        state = self.reset()

        for round_num in range(1, 6):
            print(f"Round {round_num}")
        
            player_choice = self.determine_choice()
            print(f"Player chooses: {player_choice}")
            
            state = self.step(player_choice)
            print(f"State after round {round_num}: {state}")
            
        # If dealer has less than 8 cards, draw from the deck until they have 8
        while len(self.opponent_hand) < 8 and self.deck:
            drawn_card = self.deck.pop(0)
            self.opponent_hand.append(drawn_card)
            print(f"Dealer draws: {drawn_card}")
            
        # Evaluate hands
        player_hand_strength = self.evaluate_hand(state['player_hand'])
        opponent_hand_strength = self.evaluate_hand(state['opponent_hand'])
        
        print(f"Player hand strength: {player_hand_strength}")
        print(f"Opponent hand strength: {opponent_hand_strength}")
        win_flag = 0  #1 for player win, 0 for dealer win
        if player_hand_strength < opponent_hand_strength:
            win_flag = 1  # Player wins
            print("Player wins!")
        else:
            win_flag = 0
            # Dealer wins ties
            print("Dealer wins!")

        return win_flag

    def step(self, player_choice):
        if self.round > 5:
            raise ValueError("Game over. Cannot step beyond 5 rounds.")
        
        # Player chooses cards to add to their hand
        chosen_cards = [card for card in player_choice if card in self.deck]
        if len(chosen_cards) == 0:
            raise ValueError("No valid cards chosen.")
        
        # Dealer flips cards until they find one in the player's hand
        while True:
            if not self.deck:
                raise ValueError("Deck is empty. Cannot continue game.")
            flipped_card = self.deck.pop(0)
            if flipped_card in chosen_cards:
                self.player_hand.append(flipped_card)
                break
            else:
                self.opponent_hand.append(flipped_card)
        
        # Increment round
        self.round += 1
        
        return self.get_state()
    
    def evaluate_hand(self, hand: list[Card]):
        # First, we will go through and check for flushes and straights
        # Turn the hand into 4 tuples, one for each suit, sorted in descending order of rank
        if len(hand) < 5:
            raise ValueError("Hand must have at least 5 cards.")
        suits = {suit: [] for suit in Card.suits}
        for card in hand:
            if card.suit in suits:
                suits[card.suit].append(card.value)
                
        # Sort each suit's cards in descending order
        for suit in suits:
            suits[suit].sort(reverse=True)
            
        # Check for straight flushes, making sure to handle the case where Ace can be low
        straight_flush_rank = 0
        for suit in suits:
            ranks = suits[suit]
            if len(ranks) >= 5:
                # Go through the tuple and see if we get 5 cards in a row for a straight flush
                for i in range(len(ranks) - 4):
                    if ranks[i] == ranks[i + 1] + 1 and ranks[i] == ranks[i + 2] + 2 and ranks[i] == ranks[i + 3] + 3 and ranks[i] == ranks[i + 4] + 4:
                        straight_flush_rank = max(straight_flush_rank, ranks[i])
                        break
                # Check for the special case of Ace being low
                if len(ranks) >= 5 and ranks[-1] == 2 and ranks[-2] == 3 and ranks[-3] == 4 and ranks[-4] == 5 and ranks[0] == 14:
                    straight_flush_rank = max(straight_flush_rank, 5)
        
        if straight_flush_rank > 0:
            hand_strength = 15 - straight_flush_rank
            return hand_strength

        counts = Counter(c.value for c in hand)
        # 1) Four-of-a-kind
        quads = [r for r, c in counts.items() if c >= 4]
        if quads:
            quad = max(quads)
            kicker = max(r for r in counts if r != quad)
            hand_strength = hand_lookup.get((tuple(sorted([quad]*4 + [kicker], reverse=True)), 0), 0)
            return hand_strength
        
        
        
        # 2) Full house 
        trips = sorted([r for r, c in counts.items() if c >= 3], reverse=True)
        pairs = sorted([r for r, c in counts.items() if c >= 2 and r not in trips], reverse=True)
        if trips and (len(trips) > 1 or pairs):
            three = trips[0]
            # if there are two trips, check the second highest trip vs the highest pair and choose the higher one
            if len(trips) > 1:
                pair = max(pairs[0], trips[1]) if pairs else trips[1]
            else:
                pair = pairs[0]
            hand_strength = hand_lookup.get((tuple(sorted([three]*3 + [pair]*2, reverse=True)), 0), 0)
            return hand_strength
        
       
        
        # Now check for flushes
        flush_rank = [0] * 5  # Initialize flush rank
        for suit, ranks in suits.items():
            if len(ranks) >= 5:
                flush_rank = max(flush_rank, ranks[:5]) # Take the top 5 ranks for flush
        if flush_rank[0] != 0:  # If we found a flush
            flush_rank.sort(reverse=True)
            hand_strength = hand_lookup.get((tuple(flush_rank), 1), 0)  # 1 indicates flush
            return hand_strength
        
        # Now check for straights
        unique_ranks = sorted(counts.keys(), reverse=True)
        if len(unique_ranks) > 4:
            # Add Ace as low if it exists
            if 14 in unique_ranks:
                unique_ranks.append(1)
            # Check for regular straights
            for i in range(len(unique_ranks) - 4):
                if unique_ranks[i] - 4 == unique_ranks[i + 4]:
                    if unique_ranks[i] == 5:
                        # Special case for Ace-low straight
                        hand_strength = hand_lookup.get((tuple([14, 5, 4, 3, 2]), 0), 0)
                    else:
                        hand_strength = hand_lookup.get((tuple(unique_ranks[i:i + 5]), 0), 0)
                    return hand_strength

        # 3) Three-of-a-kind
        if trips:
            three = trips[0]
            kickers = sorted((r for r in counts if r != three), reverse=True)[:2]
            hand_strength = hand_lookup.get((tuple(sorted([three]*3 + kickers, reverse=True)), 0), 0)
            return hand_strength

        # 4) Two-pair
        pair_ranks = sorted([r for r, c in counts.items() if c >= 2], reverse=True)
        if len(pair_ranks) >= 2:
            high_pair, low_pair = pair_ranks[:2]
            kicker = max(r for r in counts if r not in (high_pair, low_pair))
            hand_strength = hand_lookup.get((tuple(sorted([high_pair]*2 + [low_pair]*2 + [kicker], reverse=True)), 0), 0)
            return hand_strength

        # 5) One pair
        if pair_ranks:
            pair = pair_ranks[0]
            kickers = sorted((r for r in counts if r != pair), reverse=True)[:3]
            hand_strength = hand_lookup.get((tuple(sorted([pair]*2 + kickers, reverse=True)), 0), 0)
            return hand_strength

        # 6) High card
        top5 = sorted(counts.keys(), reverse=True)[:5]
        hand_strength = hand_lookup.get((tuple(top5), 0), 0)
        return hand_strength

    



In [135]:
#Simulate 10,000 rounds of the game:
if __name__ == "__main__":
    env = CardGameEnv()
    win_count = 0
    for _ in range(10000):
        win_flag = env.simulate_game()
        if win_flag == 1:
            win_count += 1
    print(f"Player win rate: {win_count / 10000:.2%}")

Round 1
Player chooses: [9 of 1, 10 of 1, 11 of 1, 12 of 1, 13 of 1, 14 of 1, 9 of 2, 10 of 2, 11 of 2, 12 of 2, 13 of 2, 14 of 2, 9 of 3, 10 of 3, 11 of 3, 12 of 3, 13 of 3, 14 of 3, 9 of 4, 10 of 4, 11 of 4, 12 of 4, 13 of 4, 14 of 4]
State after round 1: {'round': 2, 'deck': [14 of 2, 4 of 2, 7 of 3, 9 of 4, 3 of 3, 5 of 4, 13 of 4, 2 of 3, 3 of 2, 11 of 3, 14 of 3, 6 of 1, 5 of 3, 12 of 1, 12 of 4, 4 of 4, 3 of 1, 10 of 2, 11 of 1, 8 of 3, 9 of 3, 5 of 2, 7 of 1, 6 of 3, 2 of 4, 9 of 2, 12 of 3, 2 of 1, 10 of 1, 9 of 1, 7 of 2, 13 of 2, 6 of 4, 10 of 4, 7 of 4, 10 of 3, 14 of 1, 8 of 2, 2 of 2, 13 of 3, 12 of 2, 8 of 4, 3 of 4, 4 of 3, 11 of 4, 14 of 4, 5 of 1, 4 of 1, 11 of 2, 6 of 2, 8 of 1], 'player_hand': [13 of 1], 'opponent_hand': []}
Round 2
Player chooses: [9 of 1, 10 of 1, 11 of 1, 12 of 1, 13 of 1, 14 of 1, 9 of 2, 10 of 2, 11 of 2, 12 of 2, 13 of 2, 14 of 2, 9 of 3, 10 of 3, 11 of 3, 12 of 3, 13 of 3, 14 of 3, 9 of 4, 10 of 4, 11 of 4, 12 of 4, 13 of 4, 14 of 4]
State af

ValueError: Hand must have at least 5 cards.

In [ ]:
env = CardGameEnv()
env.reset()
# Generate a few tests to check the hand evaluation
result = env.evaluate_hand([Card(1, 14), Card(1, 13), Card(1, 12), Card(1, 11), Card(1, 10)])  # Royal Flush
assert result == 1, f"Expected 1 (Royal Flush), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 3), Card(2, 2), Card(2, 4), Card(2, 5)])  # Low Ace Straight Flush
assert result == 10, f"Expected 10 (Straight Flush), but got {result}"

result = env.evaluate_hand([Card(3, 13), Card(1, 13), Card(2, 13), Card(4, 13), Card(2, 5), Card(3, 14), Card(1, 14), Card(2, 14), Card(4, 14),])  # Four of a Kind
assert result == 11, f"Expected 11 (Four of a Kind), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 13), Card(3, 14), Card(3, 13), Card(2, 13)])  # Full House
assert result == 179, f"Expected 179 (Full House), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 3), Card(2, 2), Card(3, 4), Card(2, 5), Card(2, 7), Card(2, 9), Card(2, 10)])  # Ace Flush
assert result == 698, f"Expected 698 (Ace Flush), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 13), Card(4, 12), Card(3, 11), Card(2, 10)])  # Ace Straight
assert result == 1600, f"Expected 1600 (Straight), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 5), Card(4, 2), Card(3, 3), Card(2, 4)])  # Low Ace Straight
assert result == 1609, f"Expected 1609 (Straight), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 14), Card(3, 14), Card(4, 2), Card(2, 3)])  # Three of a Kind
assert result == 1675, f"Expected 1675 (Three of a Kind), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 14), Card(3, 13), Card(4, 13), Card(2, 3)])  # Two Pair
assert result == 2477, f"Expected 2477 (Two Pair), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 14), Card(3, 13), Card(4, 2), Card(2, 3)])  # One Pair
assert result == 3380, f"Expected 3380 (One Pair), but got {result}"

result = env.evaluate_hand([Card(2, 14), Card(2, 13), Card(3, 9), Card(4, 11), Card(2, 6)])  # High Card
assert result == 6240, f"Expected 6240 (High Card), but got {result}"